In [ ]:
from pathlib import Path

import numpy as np
from astropy.table import Table
from astropy.io import fits
from bs4 import BeautifulSoup

Definition of SIXTE's XML Instrument Configuration for XMM-Newton EPIC-PN camera
================================================================================

The current XMM files distributed by the SIXTE team include XIC (XML Instrument Configuration) files for a single PN CCD and for the whole camera. However, the last one just reproduces the size of the whole array. For a more realistic simulation, we need to define independent XIC files for each CCD.

The geometry of the camera is as follows:

![PN geometry](epicpn.png "EPIC-PN camera and spatial coordinate systems")

* CC1 corresponds to CAMCOORD1, the camera reference system with origin in the center of the camera.
* CC2 corresponds to CAMCOORD2, the camera reference system with origin in the optical axis of the corresponding telescope (XRT3).
* X/Y axis defined for each CCD corresponds to PIXCOORD1, node-oriented CCD pixel coordinate system (RAWX/RAWY in the event list).
* SC corresponds to SACCOORD, the spacecraft reference system. -Xsc axis corresponds to the viewing direction of the telescope. -Zsc axis, is North-aligned under a position angle of 0 deg, i.e. the declination axis.

See the XMM-Newton's [Calibration Access and Data Handbook](https://xmm-tools.cosmos.esa.int/external/xmm_calibration/calib/documentation/CALHB/node7.html) for more details about these coordinate systems.

According to [SIXTE's manual](https://www.sternwarte.uni-erlangen.de/research/sixte/data/simulator_manual_v1.3.14.pdf) (Sect 4.3): "The instrument coordinate system (X,Y) is defined as a planar, Cartesian system located in the focal plane behind the instrument’s optics. Its units are meter, with the origin being the point of intersection between the optical axis and the focal plane. If an attitude file is used which has the keyword ALIGNMEN set to MOTION, the X-axis of the instrument coordinate system points along the direction of motion of the pointing vector. In any other case, the X-axis points towards the celestial north pole. The Y -axis then points towards the west. If additionally a Roll Angle is defined in the attitude file, the X- and Y -axes as described before are rotated around the optical axis by this angle."

So, if I understand all this correctly, the SIXTE's instrument coordinate system (SXT) would be equivalent to the CAMCOORD2 reference system. The position of each CCD is given in the XMM calibration files (EPN_LINCOORD_0009.CCF is the most recent version as of 2021/09/24) using the center of the CCD as the reference pixel in PIXCOORD1 (RAWX0, RAWY0) in all CCD and giving its position in the CAMCOORD1 system (X0, Y0). Note that the orientation of PIXCOORD1 is the same as CAMCOORD1/2 for CCDs 7-12, but it is rotated 180 deg for CCDs 1-6. I need to convert the CC1 positions of the reference pixels into CC2 coordinates, and use that in the XIC files. The conversion factors for CC1->CC2 are given in the header of the calibration file and, as shown in the diagram above, is just a coordinates shift.

XIC files use the following parameters to define the geometry and position of a detector:
* xwidth, ywidth: Number of pixels in X_SXT and Y_SXT direction.
* xrpix, yrpix: The reference pixel.
* xrval, yrval: The location of the reference pixel in the focal plane (in m).
* xdelt, ydelt: The dimensions of the pixels (in m).

In [ ]:
filter = "thin"
add_diehlbkg = False
add_flatbkg = False
add_cfbkg = True
bkg_level = "low" # high, med, low


# We use the fullframe XIC file as a reference for defining the new files for each CCD
sixte_instruments_epicpn_path = Path("/", "storage", "sixte_instrument_files", "xmm", "epicpn")
epicpn_fullframe_xic = sixte_instruments_epicpn_path.joinpath(f"fullframe_{filter}filter.xml")

# We select the most recent xmm calibration
xmmccf_path = Path("/", "storage", "xmmccf")
epicpn_lincoord_ccf = sorted(xmmccf_path.glob("EPN_LINCOORD_*.CCF"))[-1]
lincoord = Table.read(epicpn_lincoord_ccf, hdu=1)
    
for ccd in lincoord:
    with epicpn_fullframe_xic.open() as fp:
        xic = BeautifulSoup(fp, "xml")

    xic.instrument["instrume"] = "EPN"
    xic.fov["diameter"] = 0.5
    
    xic.dimensions["xwidth"] = 64
    xic.dimensions["ywidth"] = 200
    
    xic.wcs["xrpix"] = ccd["RAWX0"] + 0.5  # The pixel coordinates in the CCF correspond to the center of the pixel
    xic.wcs["yrpix"] = ccd["RAWY0"] + 0.5

    xic.wcs["xrval"] = (ccd["X0"] - lincoord.meta["CC12_TX"]) * 1e-3  # Shift to CC2 system and convert from mm to meters
    xic.wcs["yrval"] = (ccd["Y0"] - lincoord.meta["CC12_TY"]) * 1e-3

    xic.wcs["xdelt"] = lincoord.meta["MM_PX_X"] * 1e-3  # Convert from mm to meters
    xic.wcs["ydelt"] = lincoord.meta["MM_PX_Y"] * 1e-3
    
    # TODO: check if a change in the readout direction is needed
    if ccd['CCD_ID'] <= 6:
        xic.wcs["rota"] = 180.
    else:
        xic.wcs["rota"] = 0.
        
    xic.rmf["filename"] = f"pn-{filter}-10.rmf"
    xic.arf["filename"] = f"pn-{filter}-10.arf"
    
    xic_path = Path("xml", f"fullframe_ccd{ccd['CCD_ID']:02}_{filter}filter.xml")
    
    if add_diehlbkg:
        bkgtag = xic.new_tag("phabackground", filename=f"pntffu_{bkg_level}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}diehlbkg{xic_path.suffix}"
        )
    
    elif add_flatbkg:
        bkgtag = xic.new_tag("phabackground", filename=f"pnflat_{bkg_level}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}flatbkg{xic_path.suffix}"
        )

    elif add_cfbkg:
        bkgtag = xic.new_tag("phabackground", filename=f"pnclosedfilter_{bkg_level}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}cfbkg{xic_path.suffix}"
        )

    with xic_path.open("w") as fp:
        fp.write(xic.prettify())

EPIC-MOS
----------

Now I need to define the XIC files for the MOS cameras. The geometry of the detectors is as follows:

![MOS geometry](epicmos.png "EPIC-MOS1/2 cameras and spatial coordinate systems")

For MOS1, the SIXTE reference system is oriented such as that X_SXT coincides with the Y axis of CC2, and Y_SXT is -X_CC2, i.e. SXT and CC2 are rotated 90 deg with respect to each other. Then I have to rotate each detector so the RAWX/Y matches the correct orientation.

In [ ]:
filter = "thin"
add_flatbkg = False
add_cfbkg = False
bkg_level = "low"

# We use the fullframe XIC file as a reference for defining the new files for each CCD
sixte_instruments_epicmos_path = Path("/", "storage", "sixte_instrument_files", "xmm", "epicmos")
epicmos1_fullframe_xic = sixte_instruments_epicmos_path.joinpath(f"mos1_center_{filter}filter.xml")

# We select the most recent xmm calibration
epicmos1_lincoord_ccf = sorted(xmmccf_path.glob("EMOS1_LINCOORD_*.CCF"))[-1]
lincoord = Table.read(epicmos1_lincoord_ccf, hdu=1)

for ccd in lincoord:
    if ccd["NODE_ID"] == 1:
        continue
    
    with epicmos1_fullframe_xic.open() as fp:
        xic = BeautifulSoup(fp, "xml")
        
    xic.instrument["instrume"] = "EMOS1"
    xic.instrument["telescop"] = "XMM"
    xic.fov["diameter"] = 0.5
    
    vigtag = xic.new_tag("vignetting", filename="xmm_mos1_vignet.fits")
    xic.psf.insert_after(vigtag)
    
    xic.dimensions["xwidth"] = 600
    xic.dimensions["ywidth"] = 600
    
    xic.wcs["xrpix"] = ccd["RAWX0"] + 0.5  # The pixel coordinates in the CCF correspond to the center of the pixel
    xic.wcs["yrpix"] = ccd["RAWY0"] + 0.5

    xic.wcs["xrval"] = (ccd["Y0"] - lincoord.meta["CC12_TY"]) * 1e-3  # Shift to CC2 system and convert from mm to meters
    xic.wcs["yrval"] = -(ccd["X0"] - lincoord.meta["CC12_TX"]) * 1e-3

    xic.wcs["xdelt"] = lincoord.meta["MM_PX_X"] * 1e-3  # Convert from mm to meters
    xic.wcs["ydelt"] = lincoord.meta["MM_PX_Y"] * 1e-3
    
    # TODO: check if a change in the readout direction is needed
    if ccd['CCD_ID'] == 1:
        xic.wcs["rota"] = 90.
    
    elif ccd['CCD_ID'] >= 2 and ccd['CCD_ID'] <=4:
        xic.wcs["rota"] = 0.
    
    elif ccd['CCD_ID'] >= 5:
        xic.wcs["rota"] = 180.
        
    xic.rmf["filename"] = f"mos1-{filter}-10.rmf"
    xic.arf["filename"] = f"mos1-{filter}-10.arf"
    
    xic_path = Path("xml", f"mos1_fullframe_ccd{ccd['CCD_ID']:02}_{filter}filter.xml")
    
    if add_flatbkg:
        bkgtag = xic.new_tag("phabackground", filename=f"mos1flat_{bkg_level}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}flatbkg{xic_path.suffix}"
        )
        
    elif add_cfbkg:
        bkgtag = xic.new_tag("phabackground", filename=f"mos1closedfilter_{bkg_level}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}cfbkg{xic_path.suffix}"
        )

    with xic_path.open("w") as fp:
        fp.write(xic.prettify())

For MOS2 the SIXTE reference system is oriented such as that X_SXT coincides with the -X axis of CC2, and Y_SXT is -Y_CC2, i.e. SXT and CC2 are rotated 180 deg with respect to each other. Then I have to rotate each detector so the RAWX/Y matches the correct orientation.

In [ ]:
filter = "thin"
add_flatbkg = False
add_cfbkg = False
bkg_level = "low"

# We use the fullframe XIC file as a reference for defining the new files for each CCD
sixte_instruments_epicmos_path = Path("/", "storage", "sixte_instrument_files", "xmm", "epicmos")
epicmos2_fullframe_xic = sixte_instruments_epicmos_path.joinpath(f"mos2_center_{filter}filter.xml")

# We select the most recent xmm calibration
epicmos2_lincoord_ccf = sorted(xmmccf_path.glob("EMOS2_LINCOORD_*.CCF"))[-1]
lincoord = Table.read(epicmos2_lincoord_ccf, hdu=1)

for ccd in lincoord:
    if ccd["NODE_ID"] == 1:
        continue
    
    with epicmos1_fullframe_xic.open() as fp:
        xic = BeautifulSoup(fp, "xml")
        
    xic.instrument["instrume"] = "EMOS2"
    xic.instrument["telescop"] = "XMM"
    xic.fov["diameter"] = 0.5
    
    vigtag = xic.new_tag("vignetting", filename="xmm_mos2_vignet.fits")
    xic.psf.insert_after(vigtag)
    
    xic.dimensions["xwidth"] = 600
    xic.dimensions["ywidth"] = 600
    
    xic.wcs["xrpix"] = ccd["RAWX0"] + 0.5  # The pixel coordinates in the CCF correspond to the center of the pixel
    xic.wcs["yrpix"] = ccd["RAWY0"] + 0.5

    xic.wcs["xrval"] = -(ccd["X0"] - lincoord.meta["CC12_TX"]) * 1e-3  # Shift to CC2 system and convert from mm to meters
    xic.wcs["yrval"] = -(ccd["Y0"] - lincoord.meta["CC12_TY"]) * 1e-3

    xic.wcs["xdelt"] = lincoord.meta["MM_PX_X"] * 1e-3  # Convert from mm to meters
    xic.wcs["ydelt"] = lincoord.meta["MM_PX_Y"] * 1e-3
    
    # TODO: check if a change in the readout direction is needed
    if ccd['CCD_ID'] == 1:
        xic.wcs["rota"] = 180.
    
    elif ccd['CCD_ID'] >= 2 and ccd['CCD_ID'] <=4:
        xic.wcs["rota"] = -90.
    
    elif ccd['CCD_ID'] >= 5:
        xic.wcs["rota"] = 90.
        
    xic.rmf["filename"] = f"mos2-{filter}-10.rmf"
    xic.arf["filename"] = f"mos2-{filter}-10.arf"
    
    xic_path = Path("xml", f"mos2_fullframe_ccd{ccd['CCD_ID']:02}_{filter}filter.xml")
    
    if add_flatbkg:
        bkg = "low"
        bkgtag = xic.new_tag("phabackground", filename=f"mos1flat_{bkg}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}flatbkg{xic_path.suffix}"
        )
            
    elif add_cfbkg:
        bkg = "low"
        bkgtag = xic.new_tag("phabackground", filename=f"mos2closedfilter_{bkg}_background_spectrum.fits")
        xic.arf.insert_after(bkgtag)

        xic_path = xic_path.parent.joinpath(
            f"{xic_path.stem}_{bkg_level}cfbkg{xic_path.suffix}"
        )

    with xic_path.open("w") as fp:
        fp.write(xic.prettify())

MOS vignetting
--------------

In [ ]:
# https://github.com/SamSweere/xmm-epicpn-simulator/blob/master/other/vignet_creation/create_xmm_vinget_sixte.ipynb
detector = "MOS1"
xrt_xareaef_ccf = sorted(xmmccf_path.glob(f"XRT{detector[-1]}_XAREAEF_*.CCF"))[-1]
print(xrt_xareaef_ccf)

with fits.open(xrt_xareaef_ccf) as hdu:
    # load the energy bins
    energy = hdu['VIGNETTING'].data['ENERGY']

    # Transform them to the sixte format
    energy_lo = np.array(energy[:-1]/1000.0).reshape(1, len(energy[:-1])) # E to keV 
    energy_hi = np.array(energy[1:]/1000.0).reshape(1, len(energy[1:])) # # E to keV
    
    # Create the theta step_size bins
    theta_stepsize = hdu['VIGNETTING'].header['D_THETA']
    theta_bins = hdu['VIGNETTING'].data['VIGNETTING_FACTOR'].shape[1]
    theta = np.array([i*theta_stepsize for i in range(theta_bins)], ndmin=2)

    # We do not have a phi, thus an array of zero
    phi = np.zeros((1,1))
    
    # Create the vignet array
    vignet = np.zeros((1,theta.shape[1]-1,energy_lo.shape[1])) # , dtype=[('VIGNET', '>f4')]

    ccf_vignet_data = hdu['VIGNETTING'].data['VIGNETTING_FACTOR']

    for i in range(ccf_vignet_data.shape[0]-1):
        # i indexes the theta variable
        # print(ccf_vignet_data[i][0])
        for j in range(ccf_vignet_data.shape[1]-1):
            # j intexes the energy bin variable
            # Convert to keV
            vignet[0][j][i] = ccf_vignet_data[i][j]

            
# Save vignetting into a fits file            
# The format is based on: https://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/docs/memos/cal_gen_92_021/cal_gen_92_021.html
energy_lo_col = fits.Column('ENERG_LO', format=f'{energy_lo.shape[1]}E', unit='keV', dim=f'{energy_lo.shape[1]}', array=energy_lo) 
energy_hi_col = fits.Column('ENERG_HI', format=f'{energy_hi.shape[1]}E', unit='keV', dim=f'{energy_hi.shape[1]}', array=energy_hi)
theta_col = fits.Column('THETA', format=f'{theta.shape[1]}E', unit='degree', dim=f'{theta.shape[1]}', array=theta)
phi_col = fits.Column('PHI', format=f'{phi.shape[1]}E', unit='degree', dim=f'{phi.shape[1]}', array=phi)
vignet_col = fits.Column('VIGNET', format=f'{vignet.shape[0]*vignet.shape[1]*vignet.shape[2]}E', dim=f'{vignet.shape[::-1]}', array=vignet)

columns = fits.ColDefs([energy_lo_col, energy_hi_col, theta_col, phi_col, vignet_col])

# Include the required headers
vignet_hdr = fits.Header()
vignet_hdr['HDUCLASS'] = 'OGIP'
vignet_hdr['HDUCLAS1'] = 'RESPONSE'
vignet_hdr['HDUVERS1'] = '1.0.0'
vignet_hdr['HDUCLAS2'] = 'VIGNET'
vignet_hdr['HDUVERS2'] = '1.1.0'
vignet_hdr['VERSION'] = '20171016'
vignet_hdr['MISSION'] = 'XMM'
vignet_hdr['TELESCOP'] = 'XMM'
vignet_hdr['DETNAM'] = 'XMM'
vignet_hdr['INSTRUME'] = f'E{detector}'

vignet_hdr['TUNIT1'] = 'keV'
vignet_hdr['TUNIT2'] = 'keV'
vignet_hdr['TUNIT3'] = 'degree'
vignet_hdr['TUNIT4'] = 'degree'

vignet_hdr.add_history(f'Produced by Angel Ruiz (NOA) according to data from the XMM-{detector} calibration file {xrt_xareaef_ccf.name}')

vignet_hdu = fits.BinTableHDU.from_columns(columns, header=vignet_hdr)
vignet_hdu.name = 'VIGNET'

# Create the final hdul
primary_hdu = fits.PrimaryHDU()
hdul = fits.HDUList([primary_hdu, vignet_hdu])

hdul_path = Path("xml", f"xmm_{detector.lower()}_vignet.fits")
hdul.writeto(str(hdul_path), overwrite=True)

Particle background models
============================

We created two different particle background model to be used with SIXTE for XMM-Newton simulations.

The first one is a simple flat background spectrum (as is used in Athena):

In [ ]:
# We just modify an existing fits file to makes things easier
bkg_level = "low"
spec_fits_file = Path("xml", f"pnflat_{bkg_level}_background_spectrum.fits")

with fits.open(spec_fits_file, "update") as hdu:
    hdu[1].data["CHANNEL"] = np.arange(len(hdu[1].data["RATE"]))
    hdu[1].data["RATE"] = 1. * np.ones(len(hdu[1].data["RATE"]))

In [ ]:
bkg_level = "low"
spec_fits_file = Path("xml", f"mos1flat_{bkg_level}_background_spectrum.fits")

spec = Table()
spec["CHANNEL"] = np.arange(2400, dtype=np.int32)
spec["CHANNEL"].unit = ""
spec["RATE"] = 0.1 * np.ones(2400)
spec["RATE"].unit = "cts/s/bin/m/m"

spec_hdu = fits.BinTableHDU(spec)
primary_hdu = fits.PrimaryHDU()

hdul = fits.HDUList([primary_hdu, spec_hdu])
hdul[1].name = "SPECTRUM"
hdul.writeto(spec_fits_file.as_posix(), overwrite=True)

The second one is a more realistic spectrum, based on actual data from closed-filter observations.

* I am using the merged event file produced by merging various closed filter EPIC-PN observations. The corresponding Merged Event File can be downloaded from here https://www.cosmos.esa.int/web/xmm-newton/filter-closed

* This file lists for every event the PI (Pulse Invariant channel) in units of eV. I have used this quantitity to identify the correspond EPIC PN channel using the EBOUNDS extension of an rmf (epn_e1_ff20_sdY0_v20.0.rmf - but it does not matter which specific file is used) which gives the energy limits (EMIN EMAX) of each of the 4096 channels. The PHA (Pulse Height Analyser), as far as I understand, is not the correct quantity to use for identifying channels.

* The RATE value (in counts s-1 m-2) that needs to be included in a SIXTE particle background is calculated building the histogram of the PI in the events file. I defined the histogram bins using a list of the EMIN, EMAX of the channels taken from the RMF. I have also divided by the exposure time of the merged event file (755.7e3 s) and the area in m2 of the EPIC-PN (36e-4 m2).

In [ ]:
spec_file = Path("..", "xspec", "xmm_particle_backspec_new.txt")
spec0 = Table.read(spec_file, format="ascii")

# We just modify an existing fits file to makes things easier
bkg_level = "low"
spec_fits_file = Path("xml", f"pnclosedfilter_{bkg_level}_background_spectrum.fits")


spec = Table()
spec["CHANNEL"] = np.arange(4096, dtype=np.int32)
spec["CHANNEL"].unit = ""
spec["RATE"] = 2 * spec0["col2"]
spec["RATE"].unit = "cts/s/bin/m/m"

spec_hdu = fits.BinTableHDU(spec)
primary_hdu = fits.PrimaryHDU()

hdul = fits.HDUList([primary_hdu, spec_hdu])
hdul[1].name = "SPECTRUM"
hdul.writeto(spec_fits_file.as_posix(), overwrite=True)